## steps 

* prepare data
* design data/model (input,output)
* construct loss and optmizer
* training loop
    - forwar pass: compute prediction and loss
    - backward pass: gradients
    - update weights

In [21]:
import torch 
import torch.nn as nn
import numpy as np
import matplotlib as mtl
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [22]:
#load data
df_store = datasets.load_breast_cancer()

#separate data and target
X, y = df_store.data, df_store.target

#split n_samples, n_features
n_samples, n_features = X.shape[0], X.shape[1]

#train test split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=.3,random_state=2024)

## Let torchnization begin

In [23]:
#scale
scaler = StandardScaler()
scaled_train = scaler.fit_transform(X_train)
scaled_test = scaler.transform(X_test)
#torch.from_numpy (include astype)

X_torch_train = torch.from_numpy(scaled_train.astype(np.float32))
X_torch_test = torch.from_numpy(scaled_test.astype(np.float32))
y_torch_train = torch.from_numpy(y_train.astype(np.float32))
y_torch_test = torch.from_numpy(y_test.astype(np.float32))

let's adjust the dimension of the target

In [29]:
y_torch_train = y_torch_train.view(y_torch_train.shape[0],1)
y_torch_test = y_torch_test.view(y_torch_test.shape[0],1)

## A big difference when we want to use torch
#### transform your model in a module
It brings more than the tidy advantages of modules, many pytorch utilities are design to work in modules

In [30]:
# define model as a class
class logisticreg(nn.Module):
    def __init__(self, ini_features) -> None:
        super(logisticreg, self).__init__()
        self.linear = nn.Linear(ini_features, 1)
    
    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred 
# call the model
model = logisticreg(n_features)

In [31]:
#define lr, loss function and optimizer
epochs = range(100)
lr = 0.05
l = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(),lr = lr)


In [33]:
# training loop
''' 
1. foward or pred calculation
2. loss 
3. backward ( or derivative)
4. weights update/ gradient
5. zero_grad before new iteration
(optional) print verbose to every Nth epoch
'''
for i in epochs:

    y_pred = model.forward(X_torch_train)
    loss = l(y_pred,y_torch_train)

    loss.backward()

    optimizer.step()

    optimizer.zero_grad()

    if i % 10 == 0:
        print(f'epoch: {i};  loss: {loss.item():.4f}')
    

epoch: 0;  loss: 0.6271
epoch: 10;  loss: 0.1059
epoch: 20;  loss: 0.0796
epoch: 30;  loss: 0.0713
epoch: 40;  loss: 0.0677
epoch: 50;  loss: 0.0655
epoch: 60;  loss: 0.0641
epoch: 70;  loss: 0.0630
epoch: 80;  loss: 0.0620
epoch: 90;  loss: 0.0610


In [35]:
#plot accuracy
with torch.no_grad():
    test_pred = model(X_torch_test)
    test_pred_class = test_pred.round()
    # every prediction equals to actual 'y' is summed up and divided by 'y' length
    acc = test_pred_class.eq(y_torch_test).sum() / float(y_torch_test.shape[0])
    print(f'acc: {acc:.4f}')

acc: 0.9825
